In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text=f.read()

In [ ]:
string_to_int={ch:i for i, ch in enumerate(char)}
int_to_string={i:ch for i, ch in enumerate(char)}
encode=lambda s:[string_to_int[c] for c in s]
decode=lambda s:"".join([int_to_string[c] for c in s])

In [ ]:
#Get batch function
def get_batch(split):
    data=train_data if split=='train' else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,))
    #print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)
    
    def forward(self,index,target=None):
        logit=self.token_embedding_table(index)
        if target==None:
            loss=None
        else:
         #geting shape of logits
         B, T, C=logit.shape
         logit=logit.view(B*T,C)
         target=target.view(B*T)
         loss=F.cross_entropy(logit,target)
        
        return logit, loss
    
    def generate(self,index,max_new_tokens):
        #index is (B,T) array of indices in the current content
        for _ in range(max_new_tokens):
            #get prediction
            logit, loss=self.forward(index)
            #focus only on last step
            logit=logit[:,-1,:] #becomes (B,C)
            probs=F.softmax(logit,dim=-1) #(B,C)
            index_next=torch.multinomial(probs,num_samples=1)
            #append sampled index to the running sequence
            index=torch.cat((index,index_next),dim=1)
        return index


In [ ]:
model=BigramLanguageModel(vocab_size)

In [ ]:
context=torch.zeros((1,1),dtype=torch.long,)

In [ ]:
generated_char=decode(model.generate(
    context,
    max_new_tokens=500)[0].tolist()
)
print(generated_char)


In [ ]:

optimizer=torch.optim.AdamW(model.parameters(),lr=learning_rate)

for iter in range(max_iter*20):
    xb, yb=get_batch('train')

    #Evluate loss

    logit, loss=model.forward(xb,yb)
    losst.append(loss)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss)